In [1]:
import pandas as pd

In [2]:
date_format = '%m/%d/%y'
caphis = pd.read_csv("data.csv",parse_dates=["DATE"],index_col="DATE",date_format=date_format)
caphistory = pd.DataFrame()

In [3]:
caphis.dropna(subset=["SERIAL_NUMBER"],inplace=True)

In [4]:
caphis.head(5)

,TIME,STATUS,NAME,TYPE,SOURCE,PRESENT,TECHNOLOGY,CYCLE_COUNT,VOLTAGE_MIN_DESIGN,VOLTAGE_NOW,CURRENT_NOW,CHARGE_FULL_DESIGN,CHARGE_FULL,CHARGE_NOW,CAPACITY,CAPACITY_LEVEL,MODEL_NAME,MANUFACTURER,SERIAL_NUMBER
DATE,,,,,,,,,,,,,,,,,,,
2023-07-30,22:57:13,Active,BAT0,Battery,Charging,1,Li-ion,0,11400000,11602000,13000,3684000,1814000,610000,33,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0
2023-07-30,22:53:45,Active,BAT0,Battery,Discharging,1,Li-ion,0,11400000,11511000,174000,3684000,1814000,610000,33,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0
2023-07-30,22:08:34,Active,BAT0,Battery,Discharging,1,Li-ion,0,11400000,11660000,12000,3684000,1814000,631000,34,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0
2023-07-30,21:52:34,Active,BAT0,Battery,Charging,1,Li-ion,0,11400000,11819000,1051000,3684000,1814000,358000,19,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0
2023-07-30,21:49:56,Suspended,BAT0,Battery,Not-charging,1,Li-ion,0,11400000,11397000,1000,3684000,1814000,317000,17,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0


In [5]:
caphis =  caphis[["CHARGE_FULL","CHARGE_FULL_DESIGN"]]

In [6]:
caphis["CHARGE_FULL"] = (caphis["CHARGE_FULL"]/1000).astype(int)
caphis["CHARGE_FULL_DESIGN"] = (caphis["CHARGE_FULL_DESIGN"]/1000).astype(int)

In [7]:
caphis

,CHARGE_FULL,CHARGE_FULL_DESIGN
DATE,,
2023-07-30,1814,3684
2023-07-30,1814,3684
2023-07-30,1814,3684
2023-07-30,1814,3684
2023-07-30,1814,3684
...,...,...
2023-05-03,2048,3684
2023-05-03,2048,3684
2023-05-03,2048,3684


In [8]:
curr_date=pd.to_datetime(caphis.index[-1])
week_d=curr_date+pd.DateOffset(weeks=1)
st_l = [];en_l = []; ch_l = []

In [9]:
while True:
    st_l.append(curr_date)
    en_l.append(week_d)
    ch_l.append(caphis.loc[(caphis.index>=curr_date)&(caphis.index<=week_d)]["CHARGE_FULL"].mean().astype(int))
    curr_date = curr_date+pd.DateOffset(weeks=1)
    week_d = week_d+pd.DateOffset(weeks=1)
    if(week_d>=caphis.index[0]+pd.DateOffset(days=1)):
        break

In [11]:
if curr_date<caphis.index[0]:
    week_d = caphis.index[0]
    st_l.append(curr_date)
    en_l.append(week_d)
    ch_l.append(caphis.loc[(caphis.index>=curr_date)&(caphis.index<=week_d)]["CHARGE_FULL"].mean().astype(int))
    

In [12]:
caphistory["START DATE"] = pd.Series(st_l)
caphistory["END_DATE"] = pd.Series(en_l)
caphistory["CHARGE_FULL (in mWh)"] = pd.Series(ch_l)
caphistory["CHARGE_FULL_DESIGN (in mWh)"] = caphis["CHARGE_FULL_DESIGN"][0]

In [13]:
caphistory

,START DATE,END_DATE,CHARGE_FULL (in mWh),CHARGE_FULL_DESIGN (in mWh)
0,2023-05-03,2023-05-10,2071,3684
1,2023-05-10,2023-05-17,2009,3684
2,2023-05-17,2023-05-24,2031,3684
3,2023-05-24,2023-05-31,2010,3684
4,2023-05-31,2023-06-07,1865,3684
5,2023-06-07,2023-06-14,1914,3684
6,2023-06-14,2023-06-21,2255,3684
7,2023-06-21,2023-06-28,2232,3684
8,2023-06-28,2023-07-05,2141,3684
9,2023-07-05,2023-07-12,1922,3684


In [14]:
caphistory.to_csv("battery-capacity-history.csv")